#Modelagem de Tópicos com BERT

💻 [acessar este jupyter notebook no colab](https://colab.research.google.com/drive/1foPOdLI8XrBJxXKHGeB5l8S3edigoFsG?usp=sharing)

Modelagem de tópicos é uma técnica de mineração de texto usada para descobrir automaticamente os temas ou tópicos que ocorrem em um conjunto de documentos. É particularmente útil para organizar, entender e resumir grandes coleções de texto.

## Configurações Iniciais

In [ ]:
!pip install BERTopic
!pip install sentence-transformers

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

#bibliotecas padrão do colab
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#adicionando arquivo zip de documentos
from google.colab import files
from zipfile import ZipFile
files.upload()

In [ ]:
#realizando a extração do arquivo zip
zip = ZipFile("documentos.zip")
zip.extractall()

##Tratando os documentos

O procedimento abaixo serve para colocar esses documentos em uma lista.

* O módulo `glob` em Python é utilizado para encontrar todos os nomes de caminhos correspondentes a um padrão específico. Ele permite buscar arquivos e diretórios que correspondam a um padrão usando expressões do tipo Unix, como *, ? e [ ]. No nosso caso, escontrar os arquivos *.txt.

In [ ]:
import glob
dados = []
dir = "/content/documentos/"
for arquivo in glob.iglob(dir + '*.txt', recursive=False):
  meuarquivo = open(arquivo, "r", encoding="utf8")
  content = meuarquivo.read()
  dados.append(content)
  meuarquivo.close() #é importante fechar o arquivo ao final

In [ ]:
#quantidade de documentos
len(dados)

22

In [ ]:
#realizando a leitura de algum documento
dados[7]

'Anatomia patológica\nOrigem: Wikipédia, a enciclopédia livre.\n(Redirecionado de Anatomia Patológica)\nAnatomia patológica é um ramo da patologia e da medicina que lida com o diagnóstico das doenças baseado no exame macroscópico de peças cirúrgicas e microscópicos para o exame de células e tecidos.\n\nO patologista tem ampla atuação na ciência médica. Existem patologistas dedicados preferencialmente ao desenvolvimento científico, geralmente através da patologia experimental. Outros atuam preferencialmente na sala de necrópsia, no estudo da história natural das doenças, outros ainda atuam preferencialmente em patologia cirúrgica diagnóstica e citopatologia, além de serem responsáveis pela análise e elaboração de laudos (pareceres anatomopatológicos) em exames utilizando-se a técnica de imuno-histoquímica. Aqueles que se dedicam preferencialmente à patologia diagnóstica são denominados patologistas cirúrgicos.\n\n\nÍndice\n1\tHistórico\n2\tAnatomia Patológica no Brasil\n3\tAnatomia Pato

* BERT possui uma limitação de 512 tokens para cada documento, portanto é necessário fazer o procedimento abaixo para transformar cada documento em uma lista de sentenças.

In [ ]:
sentencas = []
for item in dados:
  for t in sent_tokenize(item):
    sentencas.append(t)

In [ ]:
len(sentencas)

1504

In [ ]:
sentencas[300]

'Contudo, um bom profissional deve conter muitos outros atributos, principalmente habilidades em comunicação e de análise racional dos fatos, além de seguir um código de ética, visto que suas obras influenciam significativamente em todos os segmentos da sociedade.'

##Hiperparâmetros

Principais Hiperparâmetros:


| Hiperparâmetro    | Descrição                                           |
|-------------------|-----------------------------------------------------|
| top_n_words       | Quantas palavras terá cada tópico                   |
| n_gram_range      | N_gram (intervalo)                                  |
| min_topic_size    | Tamanho mínimo que um tópico deve ter para ser criado|
| nr_topics         | Reduz o número de tópicos iniciais para este número |
| embedding_model   | all_MiniLM-L6-v2 (sentence similarity)              |


In [ ]:
modelo = BERTopic(language="portuguese", verbose=True, embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                  top_n_words=15, min_topic_size=10, nr_topics=20,
                  vectorizer_model=CountVectorizer(ngram_range=(1,3),
                   stop_words=stopwords.words("portuguese"), min_df=10))

#all-MiniLM-L6-v2

##Rodando o modelo

Primeiro o código fara o download dos embeddings para começar a processar. O tempo de processamento depende da quantidade de documentos.

In [ ]:
%%time
topicos, probabilidades = modelo.fit_transform(sentencas)

2024-07-18 12:29:09,506 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

2024-07-18 12:34:39,678 - BERTopic - Embedding - Completed ✓
2024-07-18 12:34:39,680 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-18 12:34:58,430 - BERTopic - Dimensionality - Completed ✓
2024-07-18 12:34:58,432 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-18 12:34:58,497 - BERTopic - Cluster - Completed ✓
2024-07-18 12:34:58,499 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-18 12:34:58,702 - BERTopic - Representation - Completed ✓
2024-07-18 12:34:58,703 - BERTopic - Topic reduction - Reducing number of topics
2024-07-18 12:34:58,915 - BERTopic - Topic reduction - Reduced number of topics from 29 to 20


CPU times: user 5min 4s, sys: 12.3 s, total: 5min 17s
Wall time: 5min 49s


In [ ]:
#verificar a frequência das palavras por tópico
frequencia = modelo.get_topic_info()
frequencia.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,406,-1_grande_técnicas_além_estruturas,"[grande, técnicas, além, estruturas, engenhari...",[A introdução por Ferdinand Blum em 1894 da so...
1,0,201,0_ver_engenharia_profissional_ciência,"[ver, engenharia, profissional, ciência, sobre...",[(explicação sugerida por Enrico Coiera)\n\nAs...
2,1,187,1_maior_através_sobre_estruturas,"[maior, através, sobre, estruturas, pode, outr...",[Maior sensibilidade permite a detecção e inte...
3,2,146,2_podem_sistema_outras_estudo,"[podem, sistema, outras, estudo, exemplo, send...",[O conteúdo não verificável pode ser removido....
4,3,116,3_engenharia_engenheiro_controle_desta,"[engenharia, engenheiro, controle, desta, proc...",[A engenharia mecânica sobrepõe-se com a engen...
5,4,69,4_água_através_estudo_uso,"[água, através, estudo, uso, diversos, forma, ...",[A mineração hidráulica usa jatos de água para...
6,5,61,5_dados_áreas_área_outras,"[dados, áreas, área, outras, sobre, ainda, alé...",[A internet das coisas é um tema que se espera...
7,6,50,6_profissional_outras_áreas_área,"[profissional, outras, áreas, área, bem, atuaç...",[Índice\n1\tÁreas de Atuação\n2\tHistória\n3\t...
8,7,43,7_tipos_através_vez_origem,"[tipos, através, vez, origem, exemplo, ainda, ...",[(Novembro de 2016)\nDoença infecciosa\n\nUma ...
9,8,32,8_desenvolvimento_energia_vez_técnicas,"[desenvolvimento, energia, vez, técnicas, tecn...",[Esse profissional pode se relacionar com font...


In [ ]:
#verificando um documento em específico
modelo.get_topic(7)

[('tipos', 0.13269410264427517),
 ('através', 0.1146640766024701),
 ('vez', 0.07273204568475218),
 ('origem', 0.06483817850140566),
 ('exemplo', 0.06483817850140566),
 ('ainda', 0.06316172473994389),
 ('ver', 0.05733203830123505),
 ('podem', 0.05668665894863012),
 ('pode', 0.05051771241358393),
 ('sistema', 0.04817147198468952),
 ('tais', 0.04720987395216105),
 ('todos', 0.046160515704207164),
 ('outro', 0.046160515704207164),
 ('muitas', 0.046160515704207164),
 ('uso', 0.04423136754809172)]

##Visualização por gráficos

Utilizando atributos do módulo BERTopic.

In [ ]:
modelo.visualize_barchart(n_words=15,top_n_topics=20 )

In [ ]:
modelo.visualize_barchart(n_words=10,topics=[0,3])

In [ ]:
modelo.visualize_hierarchy(top_n_topics=15)

In [ ]:
modelo.visualize_heatmap(n_clusters=15)